Desafio aula 03 - Automação Web e busca de informações com python 

    O desafio é atualizar a planilha de vendas de produtos disponíveis em um e-commerce utilizando informações como a cotação atual do ouro e das moedas: Euro e Dólar. Porque essas informações influenciam no preço final do produto e na margem de lucro que a empresa vai ter.
    Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.
    Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver do navegador que for utilizar para a automação.

In [27]:
!pip install selenium

In [28]:
#Passo01: Importar as ferramentas dentro da biblioteca selenium

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

#Passo02: Criar o navegador e Pegar a cotação do dólar 

navegador = webdriver.Chrome()

#Entrar no site

navegador.get('https://www.google.com.br')

#Pegar a cotação do dólar
navegador.find_element('xpath','/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('Cotação Dólar')
navegador.find_element('xpath','/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cot_dolar = navegador.find_element('xpath','//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cot_dolar)

5.222049999999999


In [29]:
#Passo 03: Pegar a cotação do Euro

navegador = webdriver.Chrome()

#Entrar no site

navegador.get('https://www.google.com.br')

#Pegar a cotação do Euro
navegador.find_element('xpath','/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('Cotação Euro')
navegador.find_element('xpath','/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cot_euro = navegador.find_element('xpath','//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cot_euro)

5.114632431


In [30]:
#Passo 04: Pegar a cotação do Ouro 
import time 

navegador.get('https://www.melhorcambio.com/ouro-hoje')

cot_ouro = navegador.find_element('xpath','//*[@id="comercial"]').get_attribute('value')
cot_ouro = cot_ouro.replace(",",".")
print(cot_ouro)

time.sleep(1)

navegador.quit()

288.75


In [31]:
#Passo 05: Importar a base de dados e atualizar a base de dados

import pandas as pd 

tabela = pd.read_excel("Produtos.xlsx")
display(tabela)

#Mudar os valores da cotação na tabela com base na linha do elemento e coluna e mudar para o tipo de dado float; 

tabela.loc[tabela["Moeda"]=="Dólar","Cotação"] = float(cot_dolar)

tabela.loc[tabela["Moeda"]=="Euro","Cotação"] = float(cot_euro)

tabela.loc[tabela["Moeda"]=="Euro","Cotação"] = float(cot_euro)

# Atualizar as colunas de preço de compra e preço de venda; preço de compra = preço original * cotação 
# Preço de venda = preço de compra * margem 

tabela["Preço de Compra"] = tabela["Preço Original"] * tabela["Cotação"]

tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]

display(tabela)






,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.222050,5221.997779,1.40,7310.796891
1,Carro Renault,4500.00,Euro,5.114632,23015.845940,2.00,46031.691879
2,Notebook Dell,899.99,Dólar,5.222050,4699.792779,1.70,7989.647725
3,IPhone,799.00,Dólar,5.222050,4172.417950,1.70,7093.110515
4,Carro Fiat,3000.00,Euro,5.114632,15343.897293,1.90,29153.404857
5,Celular Xiaomi,480.48,Dólar,5.222050,2509.090584,2.00,5018.181168
6,Joia 20g,20.00,Ouro,350.000000,7000.000000,1.15,8050.000000


In [32]:
#Passo 06: Exportar a base de preços atualizados;

tabela.to_excel("Produtos Novo.xlsx",index=False)